In [1]:
import pandas as pd
import re

In [2]:
# read the data
df = pd.read_csv('../data/curated/initial.csv')
df

,Unnamed: 0,name,cost_text,coordinates,bedroom,bathroom,parking
0,0,Upstairs 2C Staley Street Brunswick VIC 3056,"$35,000 Annually","[-37.7655919, 144.9633048]",0,0,2
1,1,3502/14-16 The Esplanade St Kilda VIC 3182,"$6,000","[-37.8650177, 144.9746821]",3,3,3
2,2,9 Lansdowne Street Blairgowrie VIC 3942,"$5,000 per week","[-38.372703, 144.7856897]",3,2,3
3,3,52 Black St Brighton VIC 3186,From $5500 per week,"[-37.9159452, 144.9989003]",3,4,3
4,4,4203/35 Spring Street Melbourne VIC 3000,$4500 Per Week,"[-37.8141725, 144.9740049]",3,2,4
...,...,...,...,...,...,...,...
989,989,1 Bluff Rd Merricks Beach VIC 3926,$795,"[-38.3997481, 145.1048468]",3,2,2
990,990,1/11 Bay Street Parkdale VIC 3195,$795,"[-38.00308, 145.0810804]",3,2,2
991,991,2 Sutton Grove Richmond VIC 3121,$795 per week,"[-37.8225969, 144.990798]",2,2,1
992,992,2/261 Williams Road South Yarra VIC 3141,$795 per week,"[-37.8420314, 145.004217]",2,1,1


In [3]:
df = df.iloc[:,1:]

In [4]:
# remove some unneeded sign so that we can transfer them into number
df.loc[:,'cost_text'] = df['cost_text'].str.lower()
df.loc[:,'cost_text'] = df['cost_text'].str.replace('$','')
df.loc[:,'cost_text'] = df['cost_text'].str.replace(',','')
df.loc[:,'cost_text'] = df['cost_text'].str.replace('from','')
df['cost_text']

/tmp/ipykernel_21392/431146940.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.loc[:,'cost_text'] = df['cost_text'].str.replace('$','')


0      35000 annually
1                6000
2       5000 per week
3       5500 per week
4       4500 per week
            ...      
989               795
990               795
991      795 per week
992      795 per week
993               795
Name: cost_text, Length: 994, dtype: object

In [5]:
# convert cost data in to weekly
weekly_cost_list = []
for i in range(len(df)):
    # if already can be convert to the int
    current = df['cost_text'][i]
    try:
        weekly_cost_list.append(float(current))
    except ValueError:
        # for annually
        if "annually" in current or "pa" in current:
            new = re.findall(r"\d+\.?\d*",current)
            if len(new) == 1:
                weekly_cost_list.append(float(new[0])*7/365)
            else:
                weekly_cost_list.append(None)
        # for monthly
        elif "pm" in current or "month" in current:
            # assume an average that each month have 30 days
            new = re.findall(r"\d+\.?\d*",current)
            if len(new) == 1:
                weekly_cost_list.append(float(new[0])*7/30)
            else:
                weekly_cost_list.append(None)
        # for weekly
        elif "week" in current or "pw" in current or 'wk' in current or '/w' in current:
            new = re.findall(r"\d+\.?\d*",current)
            if len(new) == 1:
                weekly_cost_list.append(float(new[0]))
            else:
                weekly_cost_list.append(None)
        else:
            weekly_cost_list.append(None)

df.loc[:,"weekly_cost($)"] = weekly_cost_list

In [6]:
# check correctness
df.iloc[0:20,[1,6]]

,cost_text,weekly_cost($)
0,35000 annually,671.232877
1,6000,6000.000000
2,5000 per week,5000.000000
3,5500 per week,5500.000000
4,4500 per week,4500.000000
5,3775 wk 545 pn,NaN
6,3750 per week,3750.000000
7,3750.00 per week,3750.000000
8,3500.00,3500.000000
9,3500 per week,3500.000000


In [7]:
df

,name,cost_text,coordinates,bedroom,bathroom,parking,weekly_cost($)
0,Upstairs 2C Staley Street Brunswick VIC 3056,35000 annually,"[-37.7655919, 144.9633048]",0,0,2,671.232877
1,3502/14-16 The Esplanade St Kilda VIC 3182,6000,"[-37.8650177, 144.9746821]",3,3,3,6000.000000
2,9 Lansdowne Street Blairgowrie VIC 3942,5000 per week,"[-38.372703, 144.7856897]",3,2,3,5000.000000
3,52 Black St Brighton VIC 3186,5500 per week,"[-37.9159452, 144.9989003]",3,4,3,5500.000000
4,4203/35 Spring Street Melbourne VIC 3000,4500 per week,"[-37.8141725, 144.9740049]",3,2,4,4500.000000
...,...,...,...,...,...,...,...
989,1 Bluff Rd Merricks Beach VIC 3926,795,"[-38.3997481, 145.1048468]",3,2,2,795.000000
990,1/11 Bay Street Parkdale VIC 3195,795,"[-38.00308, 145.0810804]",3,2,2,795.000000
991,2 Sutton Grove Richmond VIC 3121,795 per week,"[-37.8225969, 144.990798]",2,2,1,795.000000
992,2/261 Williams Road South Yarra VIC 3141,795 per week,"[-37.8420314, 145.004217]",2,1,1,795.000000


In [8]:
df.isnull().value_counts()

name   cost_text  coordinates  bedroom  bathroom  parking  weekly_cost($)
False  False      False        False    False     False    False             862
                                                           True              132
dtype: int64